### Primary Data Cleaning

This file is to clean and process primary data which involves following steps

1. Standardizing country names
2. Clean the data
3. Converting wide format to long format
4. Merging Parts, Passenger Vehicles and Trucks
5. Writing final data to a csv to be merged with secondary data

In [ ]:
# import all the modules needed
import pandas as pd
import country_converter as coco

In [ ]:
# define root folder
ROOT_FOLDER = "."

In [ ]:
pv_export_df = pd.read_csv(ROOT_FOLDER + "/data/raw/primary/New_PV_Countries_Export_Value.csv",
                           encoding='utf-16',header=0, sep="\t")
pv_export_df.head(5)

In [ ]:
def standardize_country(countries):
    """
    Converts a list of country names or codes to their standardized short names.

    Parameters
    ----------
    countries : list or str
        A list of country names or codes, or a single country name/code, to be standardized.

    Returns
    -------
    list or str
        The standardized short names of the input countries. 
        If a single country is provided, returns a string.
        If a list is provided, returns a list of strings.

    Notes
    -----
    This function uses the `coco` library to perform the conversion. 
    The output format is the short name of each country or `not found` 
    if country name/code is not recognized.
    """
    try:
        standardized_name = coco.convert(names=countries, to='name_short')
        if standardized_name == 'not found':
            return countries
        return standardized_name
    except Exception as e:
        print(f"Error converting country name {countries}: {e}")
        return countries

In [ ]:
def clean_primary_data(df, country_col, value_col, cols_to_keep,
                       start_year=2008, end_year=2024, not_found_label='not found'):
    """
    Clean and preprocess a dataframe by standardizing country names, 
    filtering columns and rows, and melting into long format.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing data with country and year columns.
    country_col : str
        The name of the column in `df` containing country names to be standardized.
    value_col : str
        The name of the value column in the output DataFrame after melting.
    cols_to_keep : list of str
        The list of columns to keep before melting (e.g., standardized country and year columns).
    start_year : int, optional
        The first year to consider when dropping rows with missing data.
    end_year : int, optional
        The last year to consider when dropping rows with missing data.
    not_found_label : str, optional (default="not found")
        The label used to identify unrecognized countries.

    Returns
    -------
    pd.DataFrame
        A cleaned DataFrame with standardized country names, 
        a 'Year' column, and a column for data values.

    Notes
    -----
    Steps performed:
    1. Drop rows with no data between `start_year` and `end_year`.
    2. Standardize country names using `standardized_country`.
    3. Drop rows where the standardized name equals `not_found_label`.
    4. Retain only the specified columns (`cols_to_keep`).
    5. Melt the DataFrame into long format with `Year`, `standardized_country`, and values.
    """

    # Drop rows with no data in the given year range
    year_cols = [str(year) for year in range(start_year, end_year + 1)]
    df = df.dropna(subset=year_cols, how="all")

    # Standardize country names
    df['standardized_country'] = df[country_col].apply(standardize_country)

    # Drop rows where country name could not be standardized
    df = df[df['standardized_country'] != not_found_label]

    # Keep relevant columns
    df = df.loc[:, cols_to_keep]

    # Melt the dataframe to long format
    melted_df = df.melt(id_vars=['standardized_country', 'Category'],
                        var_name='year', value_name=value_col)

    return melted_df


In [ ]:
def get_cols_to_keep(start_year, end_year, country_col='standardized_country',
                     category_col='Category'):
    """
    Generate a list of columns to keep for data cleaning.

    Parameters
    ----------
    start_year : int, optional
        The first year to include in the list of columns.
    end_year : int, optional
        The last year to include in the list of columns.
    country_cols : str, optional
        Name of the country column.
    category_col : str, optional
        Name of the category column.

    Returns
    -------
    list of str
        A list of column names including 'standardized_country' and years
        from `start_year` to `end_year`.
    """
    year_cols = [str(year) for year in range(start_year, end_year + 1)]
    cols_to_keep = [country_col, category_col] + year_cols
    return cols_to_keep



In [ ]:
pv_export_df["Category"] = "Passenger Vehicles"
pv_export_df.head(5)

In [ ]:
required_cols = get_cols_to_keep(2008, 2024, 'standardized_country', 'Category')

cleaned_pv_export_df = clean_primary_data(
    pv_export_df,
    country_col='Partner',
    value_col='Export_value',
    # category_col='Category',
    cols_to_keep=required_cols
)

cleaned_pv_export_df.head(5)

## Similarly load all other export datasets into individual dataframes

In [ ]:
trucks_export_df = pd.read_csv(ROOT_FOLDER + "/data/raw/primary/MediumHeavy_Export_Countries_Value.csv",
                               encoding='utf-16',header=0, sep="\t")
trucks_export_df.head(5)

In [ ]:
trucks_export_df["Category"] = "Trucks"
cleaned_trucks_export_df = clean_primary_data(
    trucks_export_df,
    country_col='Partner',
    value_col ='Export_value',
    # category_col='Category',
    cols_to_keep=required_cols
)

cleaned_trucks_export_df.head(5)


In [ ]:
parts_export_df = pd.read_csv(ROOT_FOLDER + "/data/raw/primary/Parts_Exports_Countries.csv",
                              encoding='utf-16',header=0, sep="\t")
parts_export_df["Category"] = "Parts"
cleaned_parts_export_df = clean_primary_data(
    parts_export_df,
    country_col='Partner',
    value_col='Export_value',
    # category_col='Category',
    cols_to_keep=required_cols
)
cleaned_parts_export_df.head(5)

#### Merge Export dataframes for passenger vehicles, trucks and parts

In [ ]:
# Merge the three cleaned dataframes into one
merged_exports_df = pd.concat([
    cleaned_pv_export_df,
    cleaned_trucks_export_df,
    cleaned_parts_export_df
], ignore_index=True)

merged_exports_df.head()

In [ ]:
# Check all unique categories in the merged dataframe
merged_exports_df['year'].unique()

#### Now load csv files for PV, trucks and parts imports.
#### Then merge the cleaned dataframes to get a single dataframe for imports value

In [ ]:
# Load csv file for PV imports
pv_import_df = pd.read_csv(ROOT_FOLDER + "/data/raw/primary/New_PV_Countries_Import_Value.csv",
                           encoding='utf-16',header=0, sep="\t")

pv_import_df["Category"] = "Passenger Vehicles"

required_cols = get_cols_to_keep(2008, 2024, 'standardized_country', 'Category')

# Clean the PV imports data
cleaned_pv_import_df = clean_primary_data(
    pv_import_df,
    country_col='Partner',
    value_col='Import_value',
    # category_col='Category',
    cols_to_keep=required_cols
)

cleaned_pv_import_df.head(5)


In [ ]:
# Load csv file for parts imports
parts_import_df = pd.read_csv(ROOT_FOLDER + "/data/raw/primary/Parts_Imports_Countries.csv",
                              encoding='utf-16',header=0, sep="\t")
parts_import_df["Category"] = "Parts"

# Clean the parts imports data
cleaned_parts_import_df = clean_primary_data(
    parts_import_df,
    country_col='Partner',
    value_col='Import_value',
    # category_col='Category',
    cols_to_keep=required_cols
)

cleaned_parts_import_df.head(5)

In [ ]:
# Load csv file for truck imports
trucks_import_df = pd.read_csv(ROOT_FOLDER + "/data/raw/primary/MediumHeavy_Import_Countries_Value.csv",
                               encoding='utf-16', header=0, sep="\t")
trucks_import_df["Category"] = "Trucks"

# Clean the trucks imports data
cleaned_trucks_import_df = clean_primary_data(
    trucks_import_df,
    country_col='Partner',
    value_col='Import_value',
    # category_col='Category',
    cols_to_keep=required_cols
)

cleaned_trucks_import_df.head(5)

#### Merge Import dataframes for passenger vehicles, trucks and parts

In [ ]:
# Merge the three cleaned import dataframes into one
merged_imports_df = pd.concat([
    cleaned_pv_import_df,
    cleaned_trucks_import_df,
    cleaned_parts_import_df
], ignore_index=True)

merged_imports_df.head()

In [ ]:
# Merge Export and Import dataframes into one, keeping only rows present in both (no NaN values)
primary_df = pd.merge(
    merged_imports_df,
    merged_exports_df,
    on=["standardized_country", "year", "Category"],
    how="inner",
    suffixes=("_import", "_export")
)

primary_df.head()

In [ ]:
primary_df.to_csv(ROOT_FOLDER + "/data/processed/cleaned_primary_trade_data.csv", index=False)